In [1]:
from google.colab import drive
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Load SRGAN model from TensorFlow Hub
srgan_model = hub.load('https://tfhub.dev/captain-pool/esrgan-tf2/1')


In [3]:
# Setup VGG19 model for perceptual loss
vgg = VGG19(include_top=False, weights='imagenet')
vgg.trainable = False
feature_extractor = Model(inputs=vgg.input, outputs=vgg.get_layer('block5_conv4').output)
feature_extractor.trainable = False

def perceptual_loss(y_true, y_pred):
    # Compute VGG feature-based loss
    return tf.reduce_mean(tf.square(feature_extractor(y_true) - feature_extractor(y_pred)))

def preprocess_image(image):
    resized_image = cv2.resize(image, (224, 224))
    normalized_image = resized_image.astype(np.float32) / 255.0
    return normalized_image

def super_resolve(image):
    preprocessed_image = preprocess_image(image)
    preprocessed_image = np.expand_dims(preprocessed_image, axis=0)
    sr_image = srgan_model(preprocessed_image)
    sr_image = tf.squeeze(sr_image).numpy()
    sr_image = (sr_image.clip(0, 1) * 255.0).astype(np.uint8)
    sr_image = cv2.resize(sr_image, (image.shape[1], image.shape[0]))
    return sr_image

80134624/80134624 [==============================] - 1s 0us/step


In [4]:
# Define paths
source_dir = '/content/drive/MyDrive/aqua_data/invasive'
output_dir = '/content/drive/MyDrive/SRCNN/SRGAN/inv'

In [ ]:
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Check if the source directory is accessible and list some files
try:
    files = os.listdir(source_dir)
    print("Number of files:", len(files))
    for file in files[:10]:  # Adjust to list fewer files if the number is very large
        print(file)
except Exception as e:
    print(str(e))

# Process each image
for file in os.listdir(source_dir):
    try:
        print('Processing image:', file)
        output_path = os.path.join(output_dir, file)
        # Load image
        image_path = os.path.join(source_dir, file)
        degraded = cv2.imread(image_path)
        if degraded is None:
            continue

        # Super-resolve the image
        sr_image = super_resolve(degraded)

        # Save the super-resolved image
        cv2.imwrite(output_path, sr_image)
        print('Saved super-resolved image to', output_path)

    except Exception as e:
        print('Error processing image {}: {}'.format(file, str(e)))

Number of files: 44646
Im154 (7).png
Im152 (20).png
Im156 (1).png
Im149 (26).png
Im149 (27).png
Im153 (9).png
Im152 (21).png
Im146 (37).png
Im156 (2).png
Im154 (8).png
Processing image: Im154 (7).png
Saved super-resolved image to /content/drive/MyDrive/SRCNN/SRGAN/inv/Im154 (7).png
Processing image: Im152 (20).png
Saved super-resolved image to /content/drive/MyDrive/SRCNN/SRGAN/inv/Im152 (20).png
Processing image: Im156 (1).png
Saved super-resolved image to /content/drive/MyDrive/SRCNN/SRGAN/inv/Im156 (1).png
Processing image: Im149 (26).png
Saved super-resolved image to /content/drive/MyDrive/SRCNN/SRGAN/inv/Im149 (26).png
Processing image: Im149 (27).png
Saved super-resolved image to /content/drive/MyDrive/SRCNN/SRGAN/inv/Im149 (27).png
Processing image: Im153 (9).png
Saved super-resolved image to /content/drive/MyDrive/SRCNN/SRGAN/inv/Im153 (9).png
Processing image: Im152 (21).png
Saved super-resolved image to /content/drive/MyDrive/SRCNN/SRGAN/inv/Im152 (21).png
Processing image: I